# VisiumHD (human colorectal cancer)

We analyzed the human colorectal cancer VisiumHD dataset. This data can be obtained from 10x Genomics Data Repository (https://www.10xgenomics.com/platforms/visium/product-family/dataset-human-crc).

## 1. Import packages

In [1]:
import DiffGSP as dg
import torch
import scanpy as sc
import numpy as np
import os
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Arial']

/home/sunshuli/anaconda3/envs/DiffGSP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Read data

In [2]:
# Set parameters
data_tissue = 'Human_Colorectal_Cancer'
data_name = 'P1'
square = '016'
data_type = 'VisiumHD'
method = 'BFGS'
alpha = 0
k = 2
multiple = 8
bin_size_calculate = int(square) * 2 * multiple

In [3]:
data_path = f'/storage/sunshuli/datasets/VisiumHD/{data_tissue}/binned_outputs_{data_name}/square_{square}um'
tissue_position_file = f'{data_path}/spatial/tissue_positions.parquet'
tissue_position_csv = f'{data_path}/spatial/tissue_positions_list.csv'

if not os.path.exists(tissue_position_csv):
    tissue_position_df = pd.read_parquet(tissue_position_file)
    tissue_position_df.to_csv(tissue_position_csv, index=False, header=None)

adata = sc.read_visium(data_path, count_file='raw_feature_bc_matrix.h5')
adata.var_names_make_unique()
col, row = adata.obs['array_col'].values, -1 * adata.obs['array_row'].values
spatial = np.column_stack((col, row))
adata.obsm['spatial'] = spatial * bin_size_calculate
adata

AnnData object with n_obs × n_vars = 175434 × 37082
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

## 3. Downsampling

In [4]:
adata = dg.downsampling(adata, multiple=[multiple, multiple])
adata

Number of downsampled spots: 2704


AnnData object with n_obs × n_vars = 2704 × 37082
    obs: 'in_tissue'
    var: 'gene_ids', 'feature_types', 'genome'
    obsm: 'spatial'

## 4. Data processing, including filtering genes and identifying svgs

In [5]:
adata = adata[:, dg.prefilter_genes(adata, min_cells=3, min_counts=100)]
svgs = dg.select_svgs(adata, svg_method='gft_top', n_top=2000)
adata = adata[:, svgs[:50]]
adata

View of AnnData object with n_obs × n_vars = 2704 × 50
    obs: 'in_tissue'
    var: 'gene_ids', 'feature_types', 'genome'
    obsm: 'spatial'

## 5. Run DiffGSP (BFGS)

In [6]:
adata, optimal_solution, loss_list, constant = dg.run_diffgsp(adata, k=k, alpha=alpha, device=device, method=method,
                                                              data_type=data_type, bin_size=bin_size_calculate)
optimal_solution

Training:  34%|████▊         | 34/100 [00:06<00:11,  5.54it/s, Loss=tensor(1.9291, device='cuda:0')]


tensor([0.7202, 0.7179], device='cuda:0', dtype=torch.float64)

## 6. Run DiffGSP (16um)

In [7]:
#%% Apply to high-resolution
square = '016'

data_path = f'/storage/sunshuli/datasets/VisiumHD/{data_tissue}/binned_outputs_{data_name}/square_{square}um'
tissue_position_file = f'{data_path}/spatial/tissue_positions.parquet'
tissue_position_csv = f'{data_path}/spatial/tissue_positions_list.csv'

if not os.path.exists(tissue_position_csv):
    tissue_position_df = pd.read_parquet(tissue_position_file)
    tissue_position_df.to_csv(tissue_position_csv, index=False, header=None)
    
adata = sc.read_visium(data_path, count_file='raw_feature_bc_matrix.h5')
adata.var_names_make_unique()
adata = adata[:, svgs]
adata_raw = adata.copy()

col, row = adata.obs['array_col'].values, -1 * adata.obs['array_row'].values
spatial = np.column_stack((col, row))
bin_size = int(square) * 2
adata.obsm['spatial'] = spatial * bin_size
adata.obs['x'] = np.array(adata.obsm['spatial'][:, 0])
adata.obs['y'] = np.array(adata.obsm['spatial'][:, 1])

adata = dg.fill_adata(adata, bin_size=bin_size)
factor = bin_size / bin_size_calculate
part = int(bin_size_calculate / bin_size)
adata = dg.run_diffgsp_subgraph(adata, k=k, variable=optimal_solution.cpu().detach().numpy() * factor, array_key=['x', 'y'],
                                partition=[part, part], data_type=data_type, bin_size=2 * int(square))

adata = adata[adata_raw.obs_names, adata_raw.var_names]
adata1 = adata_raw.copy()
adata1.X = adata.X # adata1: DiffGSP result

Test: 100%|███████████████████████████████████████████| 64/64 [02:33<00:00,  2.40s/it, Finish=64/64]


## 7. Run DiffGSP (8um)

In [8]:
#%% Apply to high-resolution data
square = '008'

data_path = f'/storage/sunshuli/datasets/VisiumHD/{data_tissue}/binned_outputs_{data_name}/square_{square}um'
tissue_position_file = f'{data_path}/spatial/tissue_positions.parquet'
tissue_position_csv = f'{data_path}/spatial/tissue_positions_list.csv'

if not os.path.exists(tissue_position_csv):
    tissue_position_df = pd.read_parquet(tissue_position_file)
    tissue_position_df.to_csv(tissue_position_csv, index=False, header=None)
    
adata = sc.read_visium(data_path, count_file='raw_feature_bc_matrix.h5')
adata.var_names_make_unique()
adata = adata[:, svgs]
adata_raw = adata.copy()

col, row = adata.obs['array_col'].values, -1 * adata.obs['array_row'].values
spatial = np.column_stack((col, row))
bin_size = int(square) * 2
adata.obsm['spatial'] = spatial * bin_size
adata.obs['x'] = np.array(adata.obsm['spatial'][:, 0])
adata.obs['y'] = np.array(adata.obsm['spatial'][:, 1])

adata = dg.fill_adata(adata, bin_size=bin_size)
factor = bin_size / bin_size_calculate
part = int(bin_size_calculate / bin_size)
adata = dg.run_diffgsp_subgraph(adata, k=k, variable=optimal_solution.cpu().detach().numpy() * factor, array_key=['x', 'y'],
                                partition=[part, part], data_type=data_type, bin_size=2 * int(square))

adata = adata[adata_raw.obs_names, adata_raw.var_names]
adata1 = adata_raw.copy()
adata1.X = adata.X # adata1: DiffGSP result

Test: 100%|███████████████████████████████████████| 256/256 [10:42<00:00,  2.51s/it, Finish=256/256]
